In [9]:
import pandas as pd
import numpy as np
import urllib.request, json
from pandas.io.json import json_normalize

#все лиги по доте до сезона 2017-2018 с удалением не нужных столбцов
df = pd.read_json('https://api.opendota.com/api/leagues')
#Список 172 матчей патча 707ф
match172 = pd.read_csv('data172Matche707.csv')
match172[:1]

,match_id,avg,count,winrate,wr_lower_bound,sum,min,max,stddev
0,3534655441,10.0,1,0.5,0.23659,10,1,1,0.0


In [10]:
df[:5].drop(['banner', 'ticket', 'tier'], axis=1)
#поиск слов котоорые начинаются с 'the interww'
mask = np.array([('the inter') in x.lower() for x in df['name']]) 
name = df[mask]

In [11]:
#вытащить каждую лигу по отдельности из списка лиг с номером ID лигиц
Epicenter = df[df['name'] == 'EPICENTER: Moscow']
International_2017 = df[df['name'] == 'The International 2017']
Starladder = df[df['name'] == 'StarLadder | i-League'] 
PGL_open = df[df['name'] == 'PGL DOTA2 OPEN'] 

leag = Epicenter.append(International_2017).append(Starladder).append(PGL_open)
leag.drop(['banner', 'ticket', 'tier'], axis=1)

,leagueid,name
1213,4461,EPICENTER: Moscow
1658,5401,The International 2017
599,3902,StarLadder | i-League
1336,5616,PGL DOTA2 OPEN


In [12]:
#в списке матчей взять id одного матча
id = match172['match_id'].iloc[1]

#название сайта для одного матча
site = 'https://api.opendota.com/api/matches/' + str(id)

#Прочитаь JSON с сайта для одного матча (3536185318)
with urllib.request.urlopen(site) as url:
    data = json.loads(url.read().decode())

# нормальзовать JSON из предыдущего сайта и выбрать DF по Pick_bansq (пики и баны для матча 3536185318)
result = json_normalize(data = data, record_path='picks_bans')
result = result[result['is_pick']== True].drop(['ord', 'order'], axis=1)
result

,hero_id,is_pick,match_id,team
6,75,True,3536185318,1
7,84,True,3536185318,0
8,16,True,3536185318,0
9,82,True,3536185318,1
14,112,True,3536185318,0
15,79,True,3536185318,1
16,114,True,3536185318,0
17,52,True,3536185318,1
20,33,True,3536185318,1
21,95,True,3536185318,0


In [13]:
#Список всех героев с их id на openDota
heroes_list = pd.read_json('https://api.opendota.com/api/heroes').drop(['name', 'legs'], axis=1)
heroes_list['Disabler']=0
heroes_list['Nuker']=0
heroes_list['Carry']=0
heroes_list['Initiator']=0
heroes_list['Escape']=0
heroes_list['Durable']=0
heroes_list['Support']=0
heroes_list['Pusher']=0
heroes_list['Jungler']=0

a = -1
for x in heroes_list['roles']:
    a+=1
    for rol in x:
        if rol == 'Carry':
            heroes_list['Carry'].iloc[a] = 1
        if rol == 'Escape':
            heroes_list['Escape'].iloc[a] = 1
        if rol == 'Disabler':
            heroes_list['Disabler'].iloc[a] = 1
        if rol == 'Nuker':
            heroes_list['Nuker'].iloc[a] = 1
        if rol == 'Initiator':
            heroes_list['Initiator'].iloc[a] = 1
        if rol == 'Durable':
            heroes_list['Durable'].iloc[a] = 1
        if rol == 'Support':
            heroes_list['Support'].iloc[a] = 1
        if rol == 'Pusher':
            heroes_list['Pusher'].iloc[a] = 1
        if rol == 'Jungler':
            heroes_list['Jungler'].iloc[a] = 1

heroes_list.index = heroes_list['id']
#Необходимо чтобы данные записалсиь в основной датафрейм
heroes_list = heroes_list


C:\Sneg\Add.Program\Anaconda\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [14]:
a = heroes_list['roles']
#Перечисление всех видов героев
d = pd.Series(a[1])
for  x  in a:
        d =d.append(pd.Series(x))

d.value_counts()


Disabler     84
Nuker        84
Carry        64
Initiator    52
Escape       47
Durable      46
Support      40
Pusher       29
Jungler      15
dtype: int64

In [15]:
#Создать список имен героев из матча
id_heroes_match = np.array(result['hero_id'])
#Все аттрибуты героев, по которым оценивается герой
heroes_match_name = []
heroes_match_atack_type = []
heroes_match_primary_attr = []
heroes_match_disabler = []
heroes_match_nuker = []
heroes_match_carry = []
heroes_match_initiator = []
heroes_match_escape = []
heroes_match_durabler = []
heroes_match_support = []
heroes_match_pusher = []
heroes_match_jungler = []

#Создание массивов с данными по всем героям из данного матча
for x in id_heroes_match:
    a=-1
    for z in heroes_list['id']:
        a+=1
        if x == z:
            heroes_match_name.append(heroes_list['localized_name'].iloc[a])
            heroes_match_atack_type.append(heroes_list['attack_type'].iloc[a])
            heroes_match_primary_attr.append(heroes_list['primary_attr'].iloc[a])
            heroes_match_disabler.append(heroes_list['Disabler'].iloc[a])
            heroes_match_nuker.append(heroes_list['Nuker'].iloc[a])
            heroes_match_carry.append(heroes_list['Carry'].iloc[a])
            heroes_match_initiator.append(heroes_list['Initiator'].iloc[a])
            heroes_match_escape.append(heroes_list['Escape'].iloc[a])
            heroes_match_durabler.append(heroes_list['Durable'].iloc[a])
            heroes_match_support.append(heroes_list['Support'].iloc[a])
            heroes_match_pusher.append(heroes_list['Pusher'].iloc[a])
            heroes_match_jungler.append(heroes_list['Jungler'].iloc[a])
            
heroes = [heroes_match_name, heroes_match_atack_type, heroes_match_primary_attr, 
          heroes_match_disabler, heroes_match_nuker, heroes_match_carry, 
          heroes_match_initiator, heroes_match_escape, heroes_match_durabler, 
          heroes_match_support, heroes_match_pusher, heroes_match_jungler]

columns_heroes = ['name_heroes', 'atack_type', 'primary_attr', 
                  'Disabler', 'Nuker', 'Carry',
                  'Initiator', 'Escape', 'Durable',
                  'Support', 'Pusher', 'Jungler']

heroes_df = pd.DataFrame(heroes).T

In [16]:
#создание единой таблицы по матчу с героями и с их атрибутами
match = pd.DataFrame(result['team'])
match['hero_id'] = result['hero_id']
match.index = range(10)

match[columns_heroes] = heroes_df

match.sort_values('team')


,team,hero_id,name_heroes,atack_type,primary_attr,Disabler,Nuker,Carry,Initiator,Escape,Durable,Support,Pusher,Jungler
1,0,84,Ogre Magi,Melee,int,1,1,0,1,0,1,1,0,0
2,0,16,Sand King,Melee,str,1,1,0,1,1,0,0,0,1
4,0,112,Winter Wyvern,Ranged,int,1,1,0,0,0,0,1,0,0
6,0,114,Monkey King,Melee,agi,1,0,1,1,1,0,0,0,0
9,0,95,Troll Warlord,Ranged,agi,1,0,1,0,0,1,0,1,0
0,1,75,Silencer,Ranged,int,1,1,1,1,0,0,1,0,0
3,1,82,Meepo,Melee,agi,1,1,1,1,1,0,0,1,0
5,1,79,Shadow Demon,Ranged,int,1,1,0,1,0,0,1,0,0
7,1,52,Leshrac,Ranged,int,1,1,1,0,0,0,1,1,0
8,1,33,Enigma,Ranged,int,1,0,0,1,0,0,0,1,1


In [17]:
frame = pd.DataFrame((np.arange(6).reshape(3,2)), index=[2,0,1])

In [18]:
frame.i

AttributeError: 'DataFrame' object has no attribute 'irow'